# Ambiente

### Instalar librerías y reiniciar kernel.

In [12]:
pip install mysql-connector

In [13]:
pip install pandas

In [14]:
pip install pyarrow

### Importaciones

In [15]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import *
import json
import pyarrow.parquet as pq

### Crear acceso a engine de mysql e inspeccionar.

In [5]:
engine = create_engine('mysql+mysqlconnector://test:test123@db/test', echo = True)

In [7]:
pd.read_sql("select current_date", con=engine)

In [ ]:
inspector = inspect(engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        for column in inspector.get_columns(table_name, schema=schema):
            print("Column: %s" % column)

In [ ]:
print(engine.table_names())

# ETL 4

## Extraer

In [16]:
table = pq.read_table('data.parquet')
df = table.to_pandas()
df.head()

ArrowIOError: ignored

In [ ]:
df = pd.read_csv('time_series_covid19_confirmed_global.csv')
df.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,41814,41935,41975,42033,42159,42297,42463,42609,42795,42969,43035,43240,43468,43681,43924,44177,44363,44503,44706,44988,45174,45384,45600,45723,45844,46116,46274,46516,46718,46837,46837,47072,47306,47516,47716,47851,48053,48116,48229,48527
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,22300,22721,23210,23705,24206,24731,25294,25801,26211,26701,27233,27830,28432,29126,29837,30623,31459,32196,32761,33556,34300,34944,35600,36245,36790,37625,38182,39014,39719,40501,41302,42148,42988,43683,44436,45188,46061,46863,47742,48530
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,59527,60169,60800,61381,62051,62693,63446,64257,65108,65975,66819,67679,68589,69591,70629,71652,72755,73774,74862,75867,77000,78025,79110,80168,81212,82221,83199,84152,85084,85927,86730,87502,88252,88825,89416,90014,90579,91121,91638,92102


In [ ]:
df.dtypes

Province/State     object
Country/Region     object
Lat               float64
Long              float64
1/22/20             int64
                   ...   
12/9/20             int64
12/10/20            int64
12/11/20            int64
12/12/20            int64
12/13/20            int64
Length: 331, dtype: object

## Transformar

In [ ]:
df = df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
             var_name=["date"],
             value_name="cases")
df.head()

,Province/State,Country/Region,Lat,Long,date,cases
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [ ]:
df['date'] =  pd.to_datetime(df['date'], format='%m/%d/%y')
df.head()

,Province/State,Country/Region,Lat,Long,date,cases
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0


In [ ]:
pd.isna(df["Province/State"])

0        True
1        True
2        True
3        True
4        True
         ... 
88612    True
88613    True
88614    True
88615    True
88616    True
Name: Province/State, Length: 88617, dtype: bool

In [ ]:
u = df.select_dtypes(object)
df[u.columns] = u.apply(
    lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

## Load al engine

In [ ]:
create_table_query = pd.io.sql.get_schema(df.reset_index(), 'test.unconfirmed_transactions', con=engine)
print(create_table_query)
type(create_table_query)

In [ ]:
with open("unconfirmed_transactions.sql", "w") as text_file:
    text_file.write(create_table_query)

In [ ]:
if not engine.dialect.has_table(engine, "test.unconfirmed_transactions"):  # If table don't exist, Create.
    print("test.unconfirmed_transactions no existe")
    df.to_sql('unconfirmed_transactions', con=engine, schema="test", if_exists='append')

In [ ]:
with engine.begin() as transaction:
    transaction.execute('DELETE FROM test.unconfirmed_transactions')
    df.to_sql('unconfirmed_transactions', con=transaction, schema='test', if_exists='append', index=False)